In [40]:
import os
import re

from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyPDFLoader, PDFMinerLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore

from langsmith import traceable

from ragas import EvaluationDataset
import json
import pandas as pd

# openAI embeddings
from langchain_openai.embeddings import OpenAIEmbeddings

# vector store
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

# load api keys
from dotenv import load_dotenv
load_dotenv()

True

In [41]:
#load document
file = "../documents/2025-26_iihf_rulebook.pdf"
loader = PyPDFLoader(file)
docs = loader.load()

#drop TOC and appendix
docs_cropped = docs[15:160]

# sticth docs back together
parts = []
for i, d in enumerate(docs_cropped, start=1):
    parts.append(f"\n\n<<<PAGE {i}>>>\n{d.page_content.strip()}")

merged_text = "".join(parts)

# Create a Document and update metadata
merged_doc = [Document(
    page_content=merged_text,
    metadata={
        "source": "IIHF Rulebook 2025-26",
        "page_count": len(docs_cropped),
    }
)]


In [42]:
#print(merged_doc)

In [43]:
# for spltting
MAIN_RE = re.compile(r"RULE[ \u00A0]+(?P<main_id>\d{1,3})[ \u00A0]+(?P<main_name>[A-Z-'´’–/”“]{3,}+(?:[ \u00A0][A-Z-'´’–/”“]+)*)")
SUB_RE = re.compile(r"(?P<sub_id>\d{1,3}\.\d{1,2})[.\u00A0 ]*[ \u00A0]+(?P<sub_name>[A-Z-'´’–/”“]{2,}+(?:[ \u00A0][A-Z-'´’–/”“0-9]+)*)")


def normalize_ocr(text):
    # collapse weird spaces (regular + non-breaking)
    text = re.sub(r"[ \u00A0]+", " ", text)

    # fix error for 'penalty' which somtimes is 'penal ty' etc
    text = re.sub(r"\bPENAL\s*TY\b", "PENALTY", text)
    text = re.sub(r"\bPENAL\s*TIES\b", "PENALTIES", text)
    text = re.sub(r"\bAL\s*TERNATE\b", "ALTERNATE", text)
    
    return text


# add rule id and name to metadata
def add_rule_metadata(m, prefix):
    metadata = {
        f"{prefix}_rule": m.group(0),
        f"{prefix}_rule_id": m.group(1),
        f"{prefix}_rule_name": m.group(2),
    }
    return metadata
        




# split on rules and subrules
def slice_on_regex(docs: list[Document], pattern, prefix):
    out = []
    rx = re.compile(pattern)

    for d in docs:
        d.page_content = normalize_ocr(d.page_content)
        text = d.page_content.lstrip()
        matches = list(rx.finditer(text))
        # returns original document if no matches.
        if not matches:
            out.append(d)
            continue
        
        # For each regex match, slice from its start up to the next match (or end of text),
        # so each chunk corresponds to one rule section with its content. 
        for i, m in enumerate(matches):
            if i + 1 < len(matches):
                end = matches[i+1].start()  
            else:
                end = len(text)
            chunk = text[m.start():end]
            rule_metadata = add_rule_metadata(m, prefix)
            d.metadata.update(rule_metadata)
            out.append(Document(page_content=chunk, metadata=d.metadata))

    return out






In [44]:
main_split = slice_on_regex(merged_doc, MAIN_RE, prefix='main')
sub_split = slice_on_regex(main_split, SUB_RE, prefix='sub')

for i in sub_split:
    print(i)

page_content='1.1. RINK
Games under jurisdiction of the IIHF shall be played on an ice surface known as the “Rink” and must adhere to the dimensions and 
specifications prescribed by the IIHF and these rules.
No ice markings shall be permitted except those provided for under these rules unless express written permission has been obtained 
from the IIHF . On-ice logos must not interfere with any official ice markings provided for the proper playing of the game. 
In the interval between periods, the ice surface shall be flooded unless mutually agreed to the contrary.
' metadata={'source': 'IIHF Rulebook 2025-26', 'page_count': 145, 'main_rule': 'RULE 1 RINK', 'main_rule_id': '1', 'main_rule_name': 'RINK', 'sub_rule': '1.1. RINK', 'sub_rule_id': '1.1', 'sub_rule_name': 'RINK'}
page_content='1.2. DIMENSIONS
The official size of the rink shall be 60m long and 26m to 30m wide. The corners shall be rounded in the arc of a circle with
a radius of 7.0m to 8.50m. Any deviations from these dimens

In [63]:
main_split = slice_on_regex(merged_doc, MAIN_RE, prefix='main')
sub_split = slice_on_regex(main_split, SUB_RE, prefix='sub')


#embeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")


# vector store
vectorstore = FAISS.from_documents(documents=sub_split, embedding=embeddings)
vectorstore.save_local("../vectorstore")


parent_store = InMemoryStore()
child_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 800,
    chunk_overlap = 120
)

retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=parent_store,
    child_splitter=child_splitter,
    search_kwargs={"k": 8}
)
retriever.add_documents(sub_split)


In [64]:
# Generation step

# formatting for llm context
def format_docs(docs):
    parts = []
    for doc in docs:
        meta = {
            "main_rule_id": doc.metadata.get("main_rule_id", "N/A"),
            "main_rule_name": doc.metadata.get("main_rule_name", "N/A"),
            "sub_rule_id": doc.metadata.get("sub_rule_id", "N/A"),
            "sub_rule_name": doc.metadata.get("sub_rule_name", "N/A"),
            "source": doc.metadata.get("source", "N/A")
        }
        parts.append(f"Metadata: {meta}\n{doc.page_content.strip()}")
    return "\n\n".join(parts)


# defineing the system template
system_template = """You are an ice hockey rule assistant.

Follow these rules:
- Answer ONLY using the provided context below. If the answer is unknown or not in the context, say "I don't know".
- Use bulletpoints. After each bullet, include a citation using the metadata main rule and sub rule


Rule citation format:
[<sub_rule_id> <main_rule_name> - <sub_rule_name>]


Context (use only what is inside the markers):
---
{context}
---"""

# defining the prompt template
prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("user", "Original question: {question}"),
])


# retriever
#retriever = vectorstore.as_retriever(search_kwargs={"k": 6})



# llm
llm = init_chat_model("gpt-4o-mini", model_provider="openai", temperature=0)




multi_query_system_template = """You are an AI language model assistant. Your task is
    to generate 3 different versions of the given user
    question to retrieve relevant documents from a vector database.
    By generating multiple perspectives on the user question,
    your goal is to help the user overcome some of the limitations
    of distance-based similarity search. Provide these alternative
    questions separated by newlines. The questions are all about ice hockey.
"""

multi_query_prompt_template = ChatPromptTemplate.from_messages([
    ("system", multi_query_system_template),
    ("user", "{question}")
])


multi_retriever = MultiQueryRetriever.from_llm(retriever=retriever, llm=llm, include_original=True, prompt=multi_query_prompt_template)


rag_chain = (
    prompt_template
    | llm
    | StrOutputParser()
)

@traceable
def rag_bot(question: str):
    # retrieve
    docs = retriever.invoke(question)
    
    # build context and format docs
    context = format_docs(docs)
    
    # call rag_chain
    answer = rag_chain.invoke({"question": question, "context": context})
    
    return {"answer": answer, "documents": docs}

In [31]:
# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [65]:
# rag test
result = rag_bot("If the puck is shot before the red line, is touched after the red line and the goalie freezes it, can the defensive team change?")
#result = rag_bot("When is a player onside?")

In [66]:
print(result.get('answer'))
result.get('documents')

- No, the defensive team shall not be permitted to make any Player substitutions prior to the “face-off” in this situation. [63.8 DELAYING THE GAME - NO LINE CHANGE]


[Document(metadata={'source': 'IIHF Rulebook 2025-26', 'page_count': 145, 'main_rule': 'RULE 87 TIME-OUTS', 'main_rule_id': '87', 'main_rule_name': 'TIME-OUTS', 'sub_rule': '87.1. TIME-OUT', 'sub_rule_id': '87.1', 'sub_rule_name': 'TIME-OUT'}, page_content='87.1. TIME-OUT\nEach team shall be permitted to take one thirty-second (30s) “time-out” during the course of any game. All Players including goal -\nkeepers on the ice at the time of the “time-out” will be allowed to go to their respective benches.\nThis “time-out” must be taken during a normal stoppage of play. Only one “time-out” ("television commercial break/ power break" \nor team “time-out”), shall be permitted at any one stoppage of play. For the purpose of this rule, a “television commercial break” is \ndeemed an “official time-out” and not charged to either team.\nIIHF OFFICIAL RULE BOOK 2025/26 – SECTION 10 150\nGAME FLOW\n10 SECTION · GAME FLOW\n\n<<<PAGE 136>>>\nTABLE OF CONTENTS\nIIHF OFFICIAL RULE BOOK 2025/26 – SECTION

In [ ]:
#- No, the defensive team shall not be permitted to make any Player substitutions prior to the “face-off” in this situation. <63.1 DELAYING THE GAME> 
#- However, they can substitute a Player to replace an injured Player or when a penalty has been assessed which affects the “on-ice strength” of either team. <82.1 LINE CHANGE>

In [51]:
# setup data for ragas testing

# load file
test_questions = []
with open("eval_questions.jsonl", "r") as f:
    for line in f:
        test_questions.append(json.loads(line))
             
dataset = []
for example in test_questions:
    # extract from eval_questions
    id = example.get('id')
    question = example.get('question')
    ground_truth = example.get('ground truth')
    inner_rule = example.get('inner rule')
    
    # run the ragbot and get output answer
    rag_bot_output = rag_bot(question)
    answer = rag_bot_output.get('answer')
    
    # get context
    docs = rag_bot_output.get('documents')
    contexts = [doc.page_content for doc in docs]
    
    dataset.append({
        "user_input": question,
        "retrieved_contexts": contexts,
        "response": answer,
        "reference": ground_truth,
    })

    


In [52]:
from ragas import EvaluationDataset, evaluate
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness

evaluation_dataset = EvaluationDataset.from_list(dataset)
result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness()],
    llm=llm)
result

Evaluating:   0%|          | 0/63 [00:00<?, ?it/s]

{'context_recall': 0.8571, 'faithfulness': 0.7579, 'factual_correctness(mode=f1)': 0.6886}

In [ ]:
{'context_recall': 0.8571, 'faithfulness': 0.8571, 'factual_correctness(mode=f1)': 0.6186} # single query with top 4 docs retrieved
{'context_recall': 0.8571, 'faithfulness': 0.8608, 'factual_correctness(mode=f1)': 0.6614} # single query with top 6 docs retrieved
{'context_recall': 0.8571, 'faithfulness': 0.7579, 'factual_correctness(mode=f1)': 0.6886} # single query with top 8 docs retrieved
{'context_recall': 0.8571, 'faithfulness': 0.7540, 'factual_correctness(mode=f1)': 0.6400} # multi query with top 4 docs retrieved 
